In [1]:
"""
Feature extraction for non-interpretable approaches
(x-vector, TRILLsson, wav2vec2/HuBERT)

Input recordings assumed to be under rec_path.
Output feature files (.npy) will be saved under feat_path

@author: yiting
"""
import numpy as np
import numpy.matlib
import math
import os
import sys
from numpy import save
import pickle

import librosa
import torch
import tensorflow as tf
import tensorflow_hub as hub
from transformers import Wav2Vec2ForSequenceClassification, HubertForSequenceClassification, Wav2Vec2FeatureExtractor
from enum import Enum
import subprocess

2023-12-15 11:58:31.337294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from enum import auto


class Dataset(Enum):
    ITALIAN = auto()
    MDVR = auto()
    CZECH = auto()
    AH = auto()

# Example usage:
selected_dataset = Dataset.CZECH
option = 1 
#0 -> xvector; 1 -> trillsson;  2 -> wav2vec; 3 -> hubert

resize = True

In [3]:
if resize:

    if selected_dataset == Dataset.ITALIAN:
        dataset = "Italian"
    elif selected_dataset == Dataset.MDVR:
        dataset = "MDVR"
    elif selected_dataset == Dataset.AH:
        dataset = "AH"
    elif selected_dataset == Dataset.CZECH:
        dataset = "Czech"
    else:
        print("Unknown dataset")

    path = '../data/audios/'

    in_dir_pd = os.path.join(path, dataset+"_PD")
    out_dir_pd = os.path.join(path, dataset+"_PD/resized/")
    in_dir_hc = os.path.join(path, dataset+"_HC")
    out_dir_hc = os.path.join(path, dataset+"_HC/resized/")

    # Ensure the output directory exists
    os.makedirs(out_dir_pd, exist_ok=True)
    os.makedirs(out_dir_hc, exist_ok=True)

    for audio_file in os.listdir(in_dir_pd):
        if audio_file.endswith(".wav"):
            utt_id = os.path.basename(audio_file)
            out_path = os.path.join(out_dir_pd, utt_id)
            print(out_path)
            
            # Using subprocess to execute the sox command
            subprocess.run(["sox", os.path.join(in_dir_pd, audio_file), "-b", "16", "-r", "16k", out_path, "remix", "-"])

    for audio_file in os.listdir(in_dir_hc):
        if audio_file.endswith(".wav"):
            utt_id = os.path.basename(audio_file)
            out_path = os.path.join(out_dir_hc, utt_id)
            print(out_path)
            
            # Using subprocess to execute the sox command
            subprocess.run(["sox", os.path.join(in_dir_hc, audio_file), "-b", "16", "-r", "16k", out_path, "remix", "-"])

../data/audios/Czech_PD/resized/PD2a1_clean.wav


../data/audios/Czech_PD/resized/PD16a1_clean.wav
../data/audios/Czech_PD/resized/PD9a2_clean.wav
../data/audios/Czech_PD/resized/PD14a2_clean.wav
../data/audios/Czech_PD/resized/PD13a1_clean.wav
../data/audios/Czech_PD/resized/PD11a2_clean.wav
../data/audios/Czech_PD/resized/PD21a1_clean.wav
../data/audios/Czech_PD/resized/PD7a1_clean.wav
../data/audios/Czech_PD/resized/PD3a3_clean.wav
../data/audios/Czech_PD/resized/PD18a2_clean.wav
../data/audios/Czech_PD/resized/PD5a2_clean.wav
../data/audios/Czech_PD/resized/PD6a2_clean.wav
../data/audios/Czech_PD/resized/PD4a1_clean.wav
../data/audios/Czech_PD/resized/PD19a1_clean.wav
../data/audios/Czech_PD/resized/PD22a1_clean.wav
../data/audios/Czech_PD/resized/PD12a2_clean.wav
../data/audios/Czech_PD/resized/PD10a1_clean.wav
../data/audios/Czech_PD/resized/PD20a2_clean.wav
../data/audios/Czech_PD/resized/PD17a2_clean.wav
../data/audios/Czech_PD/resized/PD15a1_clean.wav
../data/audios/Czech_PD/resized/PD8a1_clean.wav
../data/audios/Czech_PD/res

In [4]:


def get_all_sub_segment_inds(x, fs=16e3, dur=10):
    """
        get the range of indices that can be used to run get_sub_segment()
        from the given audio signal
        
        - dur: number of seconds in a segment
    """
    N = x.shape[0] # number of samples in input signal
    N_seg = dur*fs # number of samples in a segment with the duration we want 
    ind_range = math.ceil(N/N_seg) # possible indices: 0:ind_range exclusive
    return ind_range

def get_sub_segment(x, fs=16e3, dur=10, index=0):
    """
        Get a segment of the input signal x
        
        - dur: number of seconds in a segment
        - index: index of the segment counted from the whole signal
    """
    # check if segment out of input range
    N = x.shape[0] # number of samples in input signal
    start_pt = int(index*dur*fs)
    end_pt = int(start_pt + dur*fs)
    if end_pt > N:
        end_pt = N
    
    # get segment
    seg = x[start_pt:end_pt]
    # zero padding at the end to dur if needed
    if seg.shape[0] < (dur*fs):
        pad_len = int((dur*fs)-seg.shape[0])
        seg = np.pad(seg, ((0,pad_len)), 'constant')

    return seg

# extract Paralinguistic speech embeddings
def trillsson_extraction(x,m):
    """
    get trillsson embeddings from one audio
    x: input audio (16khz)
    m = trillsson model
    """
    # normalize input
    x = x / (max(abs(x))) 

    # commented because OOM
#     ind_range = get_all_sub_segment_inds(x, fs=16e3, dur=10) # 10sec segments
#     audio_samples = np.zeros([ind_range, int(10*16e3)]) # [# of sub segments, time]
    
#     # get audios
#     for spec_ind in range(ind_range):
#         seg = get_sub_segment(x, fs=16e3, dur=10, index=spec_ind)
#         audio_samples[spec_ind,:] = seg
        
#     audio_samples = tf.convert_to_tensor(audio_samples, dtype=tf.float32)
#     embeddings = m(audio_samples)['embedding'] # num segments x 1024
#     embeddings = embeddings.numpy()

#     # average across embeddings of all sub-specs
#     features_tmp = np.mean(embeddings, axis=0) # (1024,)
#     features_tmp = features_tmp.reshape((1,1024)) # (1,1024)
    
    # divide into sub segments
    ind_range = get_all_sub_segment_inds(x, fs=16e3, dur=10) # 10sec segments
    embeddings = np.zeros(shape=(ind_range, 1024))
    for spec_ind in range(ind_range):
        seg = get_sub_segment(x, fs=16e3, dur=10, index=spec_ind)
        seg = tf.expand_dims(seg, 0) # -> tf.size [1, 160000]
        embedding = m(seg)['embedding'] # 1x1024
        embeddings[spec_ind,:] = embedding.numpy()

    # average across embeddings of all sub-specs
    features_tmp = np.mean(embeddings, axis=0) # (1024,)
    features_tmp = features_tmp.reshape((1,1024)) # (1,1024)

    return features_tmp

# x-vector extraction, given x sampled at 16kHz
def xvector_extraction(x,classifier):
    # normalize input
    x = x / (max(abs(x))) 
    x = torch.tensor(x[np.newaxis,]) # (459203,) -> torch.Size([1, 459203])

    # extract x-vectors using speechbrain
    embeddings = classifier.encode_batch(x)                        # torch.Size([1, 1, 512])
    features_tmp = embeddings.squeeze().numpy()
    features_tmp = np.reshape(features_tmp,(1, features_tmp.size)) # 1x512
    
#     # get mfccs and deltas
#     mfccs = librosa.feature.mfcc(y=x, sr=fs, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length) # n_mfcc x N
#     mfccs_delta = librosa.feature.delta(mfccs)               # n_mfcc x N
#     mfccs_delta2 = librosa.feature.delta(mfccs, order=2)     # n_mfcc x N

#     # Concatenate together as one feature vector
#     features_tmp = np.vstack((mfccs, mfccs_delta, mfccs_delta2)) # n_mfcc*3 x N
#     features_tmp = np.transpose(features_tmp)                    # N x n_mfcc*3
        
    return features_tmp

def wav2vec2_hubert_extraction(x,feature_extractor,model):
    """
    wav2vec2-base-superb-sid or hubert-base-superb-sid mean pooled hidden states extraction, 
    given x sampled at 16kHz
    """
    # normalize input
    x = x / (max(abs(x))) 
    
    # divide input into segments
    ind_range = get_all_sub_segment_inds(x, fs=16e3, dur=10) # 10sec segments
    embeddings = np.zeros(shape=(ind_range, 13, 768)) # 5 layers features x 768 dim
    for spec_ind in range(ind_range):
        seg = get_sub_segment(x, fs=16e3, dur=10, index=spec_ind)
        inputs = feature_extractor(seg, sampling_rate=16000, padding=True, return_tensors="pt")
        hidden_states = model(**inputs).hidden_states # tuple of 13 [1, frame#, 768] tensors
        for layer_num in range(13): # layer_num 0:12
            embeddings[spec_ind,layer_num,:] = hidden_states[layer_num].squeeze().mean(dim=0).detach().numpy() # [768,]

    # average across embeddings of all sub-specs
    hidden_states_list = []
    for layer_num in range(13): # layer_num 0:12
        hidden_layer_avg = np.mean(embeddings[:,layer_num,:], axis=0) # (768,)
        hidden_layer_avg = hidden_layer_avg.reshape((1,768)) # (1,768)
        hidden_states_list.append(hidden_layer_avg)

    return hidden_states_list

    # # if not dividing
    # inputs = feature_extractor(x, sampling_rate=16000, padding=True, return_tensors="pt")
    # hidden_states = model(**inputs).hidden_states
    # hidden_12 = hidden_states[-1].mean(dim=1).detach().numpy() # (1,768)
    # hidden_11 = hidden_states[11].mean(dim=1).detach().numpy()
    # hidden_10 = hidden_states[10].mean(dim=1).detach().numpy()
    # hidden_9 = hidden_states[9].mean(dim=1).detach().numpy()
    # hidden_7 = hidden_states[7].mean(dim=1).detach().numpy()
    # return hidden_12,hidden_11,hidden_10,hidden_9,hidden_7



sdir_hc = "/Users/tomas/Documents/Tese/Predictive-models-Parkinson/data/audios/"+dataset+"_HC/resized"
sdir_pd = "/Users/tomas/Documents/Tese/Predictive-models-Parkinson/data/audios/"+dataset+"_PD/resized"


wav_hc = [f for f in list(os.scandir(sdir_hc)) if f.name.endswith('.wav') ]
wav_pd = [f for f in list(os.scandir(sdir_pd)) if f.name.endswith('.wav') ]





if option == 1: #trillsson
    m = hub.KerasLayer('https://tfhub.dev/google/trillsson1/1') # select trillsson1~5

elif option == 0: # xvector
    classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

elif option == 2: # wav2vec2
    model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-sid")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-sid")

elif option == 3: # hubert
    model = HubertForSequenceClassification.from_pretrained("superb/hubert-base-superb-sid")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/hubert-base-superb-sid")

features_pd = {}
features_hc = {}

In [5]:

for wav_file in (wav_hc):
    x, fs = librosa.load(wav_file.path,sr=16000) #testt 24000
    
    if option == 0:
        features_tmp = xvector_extraction(x,classifier)
    elif option == 1:
        features_tmp = trillsson_extraction(x,m)
    else:
        hidden_states_list = wav2vec2_hubert_extraction(x,feature_extractor,model)
        features_tmp = hidden_states_list[-1]

    print("Extracting " , wav_file)
    print(features_tmp)
    
    features_hc[wav_file.name] = features_tmp

print(features_hc)



Extracting  <DirEntry 'HC22a2_clean.wav'>
[[-0.41350739 -0.10548653  0.90664183 ...  0.90972045  0.25249901
   0.60570366]]
Extracting  <DirEntry 'HC12a1_clean.wav'>
[[-0.82113965  0.08761261  1.2446187  ...  0.27625434  0.71448293
  -0.70115602]]
Extracting  <DirEntry 'HC10a2_clean.wav'>
[[-0.97431529  0.4745729   0.98703723 ...  1.41628274  0.88206334
  -0.18469048]]
Extracting  <DirEntry 'HC20a1_clean.wav'>
[[-0.81550026 -0.15904673  2.03492731 ...  0.42133991  0.88540915
   1.06211114]]
Extracting  <DirEntry 'HC4a1_clean.wav'>
[[-0.48324233  0.23025986  1.14773781 ...  1.15518934  0.19584968
  -0.09596686]]
Extracting  <DirEntry 'HC19a2_clean.wav'>
[[-0.85232276  0.07919456  1.75162941 ...  0.90480223  0.2145615
   0.76626399]]
Extracting  <DirEntry 'HC6a2_clean.wav'>
[[-0.9811945  -0.47262223  1.67583835 ...  0.234266    0.92912813
  -0.10112579]]
Extracting  <DirEntry 'HC1a1_clean.wav'>
[[-1.18545401 -0.12261377  2.14253998 ...  0.22121011  0.82688195
  -0.43128297]]
Extracting  

In [ ]:
with open("../data/test/"+dataset+str(option)+'_hc_features.pkl', 'wb') as file:
    pickle.dump(features_hc, file)

In [ ]:

for wav_file in wav_pd:
    x, fs = librosa.load(wav_file.path,sr=16000) #testt 24000

    if option == 0:
        features_tmp = xvector_extraction(x,classifier)
    elif option == 1:
        features_tmp = trillsson_extraction(x,m)
    else:
        hidden_states_list = wav2vec2_hubert_extraction(x,feature_extractor,model)
        features_tmp = hidden_states_list[-1]

    print("Extracting " , wav_file)
    print(features_tmp)
    
    features_pd[wav_file.name] = features_tmp

print(features_pd)


Extracting  <DirEntry 'VA1VSIOTLOP47M100220171331.wav'>
[[-0.61310217  0.71770376  1.44637239 ...  0.42725373  0.18747123
   1.05358878]]
Extracting  <DirEntry 'VU1sncihcio44M1606161724.wav'>
[[-1.05275066  0.40349989  0.68699102 ...  1.95433125  0.1614618
  -1.51341554]]
Extracting  <DirEntry 'VI2cdaopmoe67M2605161911.wav'>
[[-0.07961932 -0.02707214  2.14502287 ...  1.48697841  0.50294606
   2.28535616]]
Extracting  <DirEntry 'VI1rlouscsi77F2605161827.wav'>
[[-0.39539644 -0.33525253  1.30909675 ...  1.83888847  0.05279372
   0.10821521]]
Extracting  <DirEntry 'VE2ssacvhei61M1606161745.wav'>
[[ 0.02342784 -0.22578214  0.81406146 ...  2.11393404 -0.15187791
  -0.16207205]]
Extracting  <DirEntry 'VI2GLIAUDLO50F100220171303.wav'>
[[-0.33567324 -0.12859073  0.79664415 ...  2.28256917  0.14109956
  -0.38876837]]
Extracting  <DirEntry 'VU1VSIPTIOZ46M240120171932.wav'>
[[-0.76672669  0.02618179  0.48607568 ...  2.49513811  0.27192259
  -0.91728908]]
Extracting  <DirEntry 'VE2VSIPTIOZ46M240120

In [ ]:
with open("../data/non_interpretable_features/"+dataset+str(option)+'_pd_features.pkl', 'wb') as file:
    pickle.dump(features_pd, file)
